<a id="1"></a>
# <div style="padding: 20px; color: cyan; margin: 0; font-size: 30px; font-family: Arial; text-align: left; border-radius: 5px; background-color: #000000; border: 2px solid blue;"><b>Preprocessing</b></div>

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Import</b>

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_module import make_dataset
print("Setup Complete!")

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Exploring</b> data

> <span style='font-size:15px; font-family:Arial; color: #87CEFA;'><b>
Read raw data </b></span>

In [ ]:
raw_train_path = '../train.csv'
raw_test_path = '../test.csv'

raw_train_df = pd.read_csv(raw_train_path)
raw_test_df = pd.read_csv(raw_test_path)

In [ ]:
# TEST
raw_train_df.head()

> <span style='font-size:15px; font-family:Arial; color: #87CEFA;'><b>
How many rows and how many columns does the raw data have? </b></span>

In [ ]:
shape = raw_train_df.shape

In [ ]:
# TEST
print(f"Current shape: {shape}")

if shape[0] > 1000:
    print(f"Your data good!.")
else:
    print(f"Your raw data absolutely small. Please choose larger year interval.!")

> <span style='font-size:15px; font-family:Arial; color: #87CEFA;'><b>
What does each line mean? Does it matter if the lines have different meanings? </b></span>

[//]: <> (Text)
<span style="font-size:14px; font-family:Verdana; color: #ffffff;"> 
TODO: Your observation
</span>




> <span style='font-size:15px; font-family:Arial; color: #87CEFA;'><b>
Does the raw data have duplicate rows?  </b></span>

In [ ]:
index = raw_train_df.index
detectDupSeries = index.duplicated(keep='first')
num_duplicated_rows = detectDupSeries.sum()

In [ ]:
if num_duplicated_rows == 0:
    print(f"Your raw data have no duplicated line.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Your raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate your raw data.!")

In [ ]:
# De-deduplicate your raw data
if num_duplicated_rows > 0:
    raw_train_df = raw_train_df.drop_duplicates(keep='first')
    print('Shape after de-deduplicate: ', raw_train_df.shape)

> <span style='font-size:15px; font-family:Arial; color: #87CEFA;'><b>
What data type does each column currently have? Are there any columns whose data types are not suitable for further processing?  </b></span>

In [ ]:
dtypes = raw_train_df.dtypes

In [ ]:
# TEST
dtypes

[//]: <> (Text)
<span style="font-size:14px; font-family:Verdana; color: #ffffff;"> 
TODO: Your observation
</span>

> <span style='font-size:15px; font-family:Arial; color: #87CEFA;'><b>
For each column with numeric data type, how are the values distributed? </b></span>

In [ ]:
num_col_info_train_df = raw_train_df.select_dtypes(exclude=['object', 'bool'])

def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

num_col_info_train_df = num_col_info_train_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])

In [ ]:
num_col_info_train_df

In [ ]:
# YOUR CODE HERE (Optional)
# You can use Matplotlib to visualize this table for more intuitive

In [ ]:
print(dict(num_col_info_train_df.iloc[0]))


> <span style='font-size:15px; font-family:Arial; color: #87CEFA;'><b>
For each column with a non-numeric data type, how are the values distributed? </b></span>


[//]: <> (Text)
<span style="font-size:14px; font-family:Arial; color: #ffffff;"> 

For non-numeric columns in a Pandas DataFrame, calculate the following:

- Percentage of missing values: This is the percentage of values in the column that are missing, from 0 to 100.
- Number of unique values: This is the number of different values in the column, excluding missing values. For categorical columns, this is the same as the number of categories.
- Percentage of each value: This is the percentage of each unique value in the column, sorted by decreasing percentage. Missing values are excluded from this calculation.

To calculate the percentage of each value, you can use a dictionary to store the results, where the key is the value and the value is the percentage. For categorical columns, the method is the same.

Save the results to a Pandas DataFrame called cat_col_info_df, with the following columns:

- Column name: The name of the non-numeric column in the original DataFrame (raw_df)
- missing_ratio: The percentage of missing values in the column
- num_values: The number of unique values in the column, excluding missing values
- value_ratios: A dictionary of the percentage of each unique value in the column, sorted by decreasing percentage

To improve readability, round all values to one decimal place using the .round(1) method.

</span>

In [ ]:
pd.set_option('display.max_colwidth', 100) # For clearly
pd.set_option('display.max_columns', None) # For clearly

In [ ]:
cat_col_info_df = raw_train_df.select_dtypes(include='object')

def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def num_values(s):
    s = s.astype(str).str.split(';')
    s = s.explode()
    return len(s.value_counts())

def value_ratios(s):
    s = s.astype(str).str.split(';')
    s = s.explode()
    totalCount = (~s.isna()).sum()
    return ((s.value_counts()/totalCount*100).round(1)).to_dict()

cat_col_info_df = cat_col_info_df.agg([missing_ratio, num_values, value_ratios])
cat_col_info_df

In [ ]:
# YOUR CODE HERE (Optional)
# You can use Matplotlib to visualize this table for more intuitive

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Cleaning </b> data

[//]: <> (Text)
<span style="font-size:14px; font-family:Arial; color: #ffffff;"> 

After identifying the basic statistical numbers that describe your data, we further need to determine the features that have a large number of missing values. Such features are not useful for the analysis stage and must be removed from the dataset.

Depending on your goals, the threshold for "large" can be defined. Usually, if the percentage of missing values is greater than 75%, the column is dropped from the dataframe and an updated dataframe is returned.

</span>

In [ ]:
def drop_missing_features(df: pd.DataFrame, missing_lst: dict = dict(num_col_info_train_df.iloc[0]), threshold: float = 75.0) -> pd.DataFrame:
    """Drop missing features from Pandas dataframe base on given threshold.

    Args:
        df (pd.DataFrame): Input dataframe for processing.
        missing_lst (dict, optional): Dict missing value rate which calculated from previous stage. Defaults to dict(num_col_info_df.iloc[0]).
        threshold (float, optional): User threshold for dropping. Defaults to 75.0.

    Returns:
        pd.DataFrame: Updated dataframe
    """
    if (df is None) or (missing_lst is None) or (threshold is None):
        print(f"[LOG] Invalid.")
        raise ValueError
    
    # Create a copy of the dataframe to avoid changing the original
    df_cp = df.copy()
    
    # List to maintain the columns to drop
    cols_to_trim = []
    
    for key, value in missing_lst.items():
        if float(value) > threshold:
            cols_to_trim.append(key)
            
    if len(cols_to_trim) > 0:
        # drop the columns identified using the dataframe drop() method
        df_cp = df_cp.drop(columns=cols_to_trim)
        print("[LOG] Dropped column(s): " + " ".join(cols_to_trim))
    else:
        print("[LOG] Have no column(s) to trim.")
        
    return df_cp

In [ ]:
raw_train_df = drop_missing_features(raw_train_df, dict(num_col_info_train_df.iloc[0]))

In [ ]:
# TEST
raw_train_df.head()


[//]: <> (Text)
<span style="font-size:14px; font-family:Arial; color: #ffffff;"> 

After remove features which have large missing values, our dataframe still have missing values. So that, we need to fill these missing values so that they can be used in analysis.

</span>

In [ ]:
def filling_missing_value(df: pd.DataFrame) -> pd.DataFrame:
    """Filling missing features from Pandas dataframe
    
    The first, you need to replace all None values with NaN, fillna only works on nans.
    
    After that, you replace all NaN values with the mean of the column values.

    Args:
        df (pd.DataFrame): Input dataframe for processing.

    Returns:
        pd.DataFrame: Updated dataframe
    """
    if (df is None):
        print(f"[LOG] Invalid.")
        raise ValueError
    
    # Create a copy of the dataframe to avoid changing the original
    df_cp = df.copy()
    
    # Get the list of columns in the dataframe
    cols_list = list(df_cp.columns)
    
    # Replace all None values with NaN, fillna only works on nans
    df_cp.fillna(value=np.nan, inplace=True)
    
    # Replace all NaN values with the mean of the column values
    for col in cols_list:
        if df_cp[col].dtype in ['object', 'bool']:
            df_cp[col].fillna((df_cp[col].mode()[0]), inplace=True)
        elif df_cp[col].dtype in ['int64', 'float64']:
            df_cp[col].fillna((df_cp[col].mean()), inplace=True)
        
    return df_cp
    

In [ ]:
raw_train_df = filling_missing_value(df=raw_train_df)

In [ ]:
# TEST
raw_train_df.select_dtypes(exclude=['object', 'bool']).agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])

In [ ]:
raw_train_df.select_dtypes(include=['object', 'bool']).agg([missing_ratio, num_values, value_ratios])

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Transformating </b> data

> <span style='font-size:15px; font-family:Arial; color: #87CEFA;'><b>
Is the collected data reasonable? </b></span>

<a id="1.2"></a>
<h2 style=" font-family: Arial; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: #008B8B; padding: 15px; border-top: 2px solid blue; border-bottom: 2px solid blue;
"><b>Saving </b> data

In [ ]:
print(f"Total number of features: {raw_train_df.shape[1]}")
raw_train_df.dtypes

In [ ]:
# YOUR CODE HERE
save_name_lst = ['train', 'test']
for name in save_name_lst:
    print(f"Save {name} to ../data/processed/{name}_processed.csv")
    raw_train_df.to_csv("../data/processed/" + name + "_processed.csv", index=False)